# **Relação entre gastos militares das nações e seu impacto econômico: positivo ou negativo?**

---


# 1) Importando e tratando os dados

In [68]:
import pandas as pd

# importando as bases de dados de seus respctivos caminhos
sipri_data = pd.read_excel(
    r'c:\Users\Meus Documentos\Downloads\SIPRI-Milex-data-2014-2024.xlsx',
    sheet_name=[
        'Constant (2023) US$', 'Current US$', 'Share of GDP',
        'Per capita', 'Share of Govt. spending'
    ]
)
fmi_data = pd.read_csv(r'c:\Users\Meus Documentos\Downloads\dataset_2025-06-04T01_01_07.768164706Z_DEFAULT_INTEGRATION_IMF.RES_WEO_6.0.0.csv')


# a) Tratando a base sipri_data

In [ ]:
# passando os dados para o formato long com .melt(), de modo que um ano esteja associado a mais de uma variável
constant_exp_long = sipri_data['Constant (2023) US$'].melt(id_vars=['Country'], var_name='Year', value_name='militar_expenses')
current_exp_long = sipri_data['Current US$'].melt(id_vars=['Country'], var_name='Year', value_name='militar_expenses_cur')
gdp_share_long = sipri_data['Share of GDP'].melt(id_vars=['Country'], var_name='Year', value_name='expenses_share_of_gdp')
percap_exp_long = sipri_data['Per capita'].melt(id_vars=['Country'], var_name='Year', value_name='expenses_per_capita')
govspend_long = sipri_data['Share of Govt. spending'].melt(id_vars=['Country'], var_name='Year', value_name='expenses_share_of_gov')

# definindo e aplicando a função clean para converter valores não numéricos e eliminá-los se nulos
def clean(df, value_col):
    df[value_col] = pd.to_numeric(df[value_col], errors='coerce')
    df['Year'] = df['Year'].astype(int)
    return df

constant_exp_long = clean(constant_exp_long, 'militar_expenses')
current_exp_long = clean(current_exp_long, 'militar_expenses_cur')
gdp_share_long = clean(gdp_share_long, 'expenses_share_of_gdp')
percap_exp_long = clean(percap_exp_long, 'expenses_per_capita')
govspend_long = clean(govspend_long, 'expenses_share_of_gov')

# mergindo cada uma delas sucessivamente até termos um único dataframe
military_expenses = constant_exp_long.merge(current_exp_long, on=['Country', 'Year'], how='outer')
military_expenses = military_expenses.merge(gdp_share_long, on=['Country', 'Year'], how='outer')
military_expenses = military_expenses.merge(percap_exp_long, on=['Country', 'Year'], how='outer')
military_expenses = military_expenses.merge(govspend_long, on=['Country', 'Year'], how='outer')

# remover linhas nulas e verificando se funcionou com .sum()
military_expenses = military_expenses.dropna(subset=[
    'militar_expenses', 'militar_expenses_cur',
    'expenses_share_of_gdp', 'expenses_per_capita',
    'expenses_share_of_gov'
])

print(military_expenses.isna().sum())

# verificando nosso df final
print(military_expenses.info())
print(military_expenses.head())

# b) Tratando a base fmi_data

In [ ]:
# 

                             COUNTRY  \
0                         Montenegro   
1                            Morocco   
2                 Nauru, Republic of   
3                 Nauru, Republic of   
4  Marshall Islands, Republic of the   

                                        INDICATOR     Year           fmi_value  
0    Net debt, General government, Percent of GDP  DATASET  IMF.RES:WEO(6.0.0)  
1  Gross debt, General government, Percent of GDP  DATASET  IMF.RES:WEO(6.0.0)  
2                               Unemployment rate  DATASET  IMF.RES:WEO(6.0.0)  
3    Net debt, General government, Percent of GDP  DATASET  IMF.RES:WEO(6.0.0)  
4    Net debt, General government, Percent of GDP  DATASET  IMF.RES:WEO(6.0.0)  
